In [2]:
import os
import re
import sqlite3
import numpy as np
import pandas as pd
from pandas import DataFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.float_format', str)

# Load 'funds_df' DataFrame from the specified file
funds_file_path = os.path.join("Enter_Funds_For_FSA.xlsx")
funds_df = pd.read_excel(funds_file_path, sheet_name='Enter Funds')

#InformationCoefficient = 1.6[BattingAverage - 1/(1 + SluggingRatio)]
#InformationRatio = InformationCoefficient * (InfomationCoefficient)**1/2

In [3]:
funds_df.head()

,Funds
0,Bank of India ELSS Tax Saver - Eco - Growth
1,360 ONE Flexicap Fund - Regular - Growth
2,360 ONE Focused Equity Fund - Regular - Growth
3,360 ONE Quant Fund - Regular - Growth
4,Aditya Birla Sun Life Banking and Financial Se...


In [4]:
# function
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='openpyxl')
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0, index=False)
    writer.close()

def getData(db, insert=True):
  '''
    If creating database from scratch, use the excel file which has complete historical data)
  '''

  if insert:
    # Define the path to the Excel file
    file_uploads = ["data/Holdings Historical 360 to DSP.xlsx",
      "data/Holdings Historical Edelweiss to Invesco.xlsx",
      "data/Holdings Historical ITI to PPFAS.xlsx",
      "data/Holdings Historical Quant to Zerodha.xlsx"]

    # Loop through each Excel file in 'file_uploads'
    for i in file_uploads:
      dfs = pd.read_excel(i,engine="openpyxl",skiprows=4)
      dfs.to_sql('df_table', con=db, if_exists='append')

    cols = ['CD_ISIN No', 'Bmonth_Month End', 'Bmonth_Close']
    colRename = {'CD_ISIN No': 'Company ISIN', 'Bmonth_Month End': 'Numeric Date', 'Bmonth_Close': 'Price'}
    companyDataDf = pd.read_excel("data/Company_Data.xlsx",engine="openpyxl", usecols=cols)
    companyDataDf_ = companyDataDf.rename(columns=colRename)
    companyDataDf_.to_sql('company_table', con=db, if_exists='append')
    
    return db.cursor()
  else:
    return db.cursor()

def calculate_information_ratio(mergedDf, fundInfo):

  numWins = 0
  numLosses = 0
  absoluteGain = 0
  absoluteLoss = 0

  # Get the breadth of the fund
  stockList = mergedDf['Company Name'].unique()
  breadth = len(stockList)
  print("Breadth:",breadth)
  if breadth == 0:
    return fundInfo, 4
  fundInfo.append(breadth)

  for i in stockList:
    stockDf = mergedDf[mergedDf['Company Name'] == i]
    stockDf["diff_tmp"] = stockDf["No Of Shares"].diff()
    stockDf["ret_tmp"] = stockDf["Price"] * stockDf["diff_tmp"]
    stockDf['ret_tmp'].iloc[0] = stockDf['Price'].iloc[0] * stockDf['No Of Shares'].iloc[0]

    # We are assuming the the portfoli manager has exited the remaining shares at the end of the analysis period
    lastExit = stockDf['Price'].iloc[-1] * stockDf['No Of Shares'].iloc[-1]

    buySum = stockDf[stockDf['ret_tmp']>=0]['ret_tmp'].sum()

    sellSum = np.abs(stockDf[stockDf['ret_tmp']<=0]['ret_tmp']).sum() + lastExit

    if buySum < sellSum:
      numWins+=1
      absoluteGain += round((sellSum-buySum),3)
    else:
      numLosses+=1
      absoluteLoss += round((sellSum-buySum),3)

  if numWins == 0:
    return fundInfo, 0
  else:
    averageGain = abs(absoluteGain)/numWins
    # averageGain = abs(absoluteGain)
  if numLosses == 0:
    return fundInfo, 1
  else:
    averageLoss = abs(absoluteLoss)/numLosses
    # averageLoss = abs(absoluteLoss)

  fundInfo.append(numWins)
  fundInfo.append(numLosses)

  #Pain-to-Gain Ratio
  pain_gain_ratio = abs(absoluteGain/absoluteLoss)

  # Calculate the batting average
  battingAverage = numWins/breadth
  fundInfo.append(round(battingAverage,3))

  # Calculate the slugging ratio
  sluggingRatio = averageGain/averageLoss

  fundInfo.append(round(sluggingRatio,3))

  informationCoefficient = 1.6*(battingAverage - (1/(1+sluggingRatio)))
  fundInfo.append(round(informationCoefficient,3))

  informationRatio = informationCoefficient * (np.sqrt(breadth))
  fundInfo.append(round(informationRatio,3))
  fundInfo.append(round(pain_gain_ratio,3))

  # print("Breadth:", breadth)
  # print("Total Wins:", numWins)
  # print("Total Losses:", numLosses)
  # print("Batting Average:",battingAverage)
  # print("Slug Ratio:",sluggingRatio)
  # print("Info Coefficient:",informationCoefficient)
  # print("Info Ratio:", informationRatio)

  return fundInfo, 3

In [5]:
ir_dict = []
only_wins = []
only_losses = []
dont_exist = []
today = datetime.now()

db = sqlite3.connect('/home/rolf/Documents/trustPlutus/infoRatio/MutualFundData.db')
cursor = getData(db, insert=False)


In [6]:
for i in funds_df['Funds']:

  fundInfo = []
  fundInfo.append(i)
  # fundQuery = cursor.execute("SELECT * FROM df_table WHERE `Scheme Name` = ? ", (i,)).fetchall()
  fundQuery = cursor.execute("SELECT * FROM df_table").fetchall()
  df = DataFrame(fundQuery)
  df.columns = [i[0] for i in cursor.description]

  # Add 'Numeric Date' column by applying the convert_to_numeric_date function on the 'Port Date' column
  df['Numeric Date'] = df['Port Date'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y").strftime("%Y%m")).astype(np.int64)

  companyDataQuery = cursor.execute("SELECT * FROM company_table").fetchall()
  companyDataDf_ = DataFrame(companyDataQuery)
  companyDataDf_.columns = [i[0] for i in cursor.description]

  print("c_data ",companyDataDf_.shape)

  # Merge single_fund_df with ace_df_renamed on 'Company ISIN' and 'Numeric Date'
  # Use 'left' join to keep all rows from single_fund_df and fill in 'Price' where matches are found
  mergedDf = pd.merge(df, companyDataDf_, on=['Company ISIN', 'Numeric Date'], how='left')
  mergedDf["No Of Shares"]= mergedDf["No Of Shares"].replace('--', None, regex=True)
  mergedDf = mergedDf[mergedDf['Price'].notna()]
  mergedDf = mergedDf[mergedDf["No Of Shares"].notna()]
  mergedDf['Port Date_'] = pd.to_datetime(mergedDf['Port Date'])
  mergedDf = mergedDf.sort_values(by='Port Date_')
  mergedDf["No Of Shares"] = mergedDf["No Of Shares"].astype(int)

  end = pd.Timestamp('2022-12-31')
  start = pd.Timestamp('2021-12-31')

  mergedDf = mergedDf.loc[mergedDf['Port Date_'].between(start, end)]
  print(mergedDf["Port Date_"].max())
  print(mergedDf["Port Date_"].min())
  print("Fund:",i)
  print("merged shape:",mergedDf.shape)

  mergedDf = mergedDf[mergedDf['Scheme Name'] == i]
  print("merged shape:",mergedDf.shape)
  
  # mergedDf.to_excel(f"{i}.xlsx")

  fundInfo, type_ = calculate_information_ratio(mergedDf, fundInfo)
  if type_ == 0:
    only_losses.append(fundInfo)
  elif type_ == 1:
    only_wins.append(fundInfo)
  elif type_ == 4:
    dont_exist.append(fundInfo)
  else:
    ir_dict.append(fundInfo)
  print("-------------------")

c_data  (575898, 4)
2022-12-31 00:00:00
2021-12-31 00:00:00
Fund: Bank of India ELSS Tax Saver - Eco - Growth
merged shape: (217002, 15)
merged shape: (691, 15)
Breadth: 103
-------------------
c_data  (575898, 4)
2022-12-31 00:00:00
2021-12-31 00:00:00
Fund: 360 ONE Flexicap Fund - Regular - Growth
merged shape: (217002, 15)
merged shape: (0, 15)
Breadth: 0
-------------------
c_data  (575898, 4)
2022-12-31 00:00:00
2021-12-31 00:00:00
Fund: 360 ONE Focused Equity Fund - Regular - Growth
merged shape: (217002, 15)
merged shape: (413, 15)
Breadth: 41
-------------------
c_data  (575898, 4)
2022-12-31 00:00:00
2021-12-31 00:00:00
Fund: 360 ONE Quant Fund - Regular - Growth
merged shape: (217002, 15)
merged shape: (424, 15)
Breadth: 59
-------------------
c_data  (575898, 4)
2022-12-31 00:00:00
2021-12-31 00:00:00
Fund: Aditya Birla Sun Life Banking and Financial Services Fund - Regular - Growth
merged shape: (217002, 15)
merged shape: (434, 15)
Breadth: 41
-------------------
c_data  (5

In [7]:
df = pd.DataFrame(ir_dict, columns = ['Fund Name', 'Breadth', 'Wins', 'Loss', "Batting Average", "Slugging Ratio", "Information Coefficient", "Information Ratio","Pain/Gain Ratio"])
only_win_df = pd.DataFrame(only_wins, columns = ['Fund Name', 'Breadth'])
only_loss_df = pd.DataFrame(only_losses, columns = ['Fund Name', 'Breadth'])
dont_exist_df = pd.DataFrame(dont_exist, columns = ['Fund Name'])

In [8]:
dont_exist_df.head()

,Fund Name
0,360 ONE Flexicap Fund - Regular - Growth


In [21]:
# list of dataframes and sheet names
dfs = [df, only_win_df, only_loss_df,dont_exist_df]
sheets = ['Main','Only Wins','Only Losses',"Don't Exist"]

name =  today.strftime("%Y-%m-%d")+".xlsx"
# run function
dfs_tabs(dfs, sheets, name)

In [9]:
df.head()

,Fund Name,Breadth,Wins,Loss,Batting Average,Slugging Ratio,Information Coefficient,Information Ratio,Pain/Gain Ratio
0,Bank of India ELSS Tax Saver - Eco - Growth,103,42,61,0.408,1.788,0.079,0.798,1.231
1,360 ONE Focused Equity Fund - Regular - Growth,41,17,24,0.415,1.522,0.029,0.185,1.078
2,360 ONE Quant Fund - Regular - Growth,59,23,36,0.39,1.823,0.057,0.438,1.165
3,Aditya Birla Sun Life Banking and Financial Se...,41,23,18,0.561,3.19,0.516,3.302,4.077
4,Aditya Birla Sun Life Business Cycle Fund - Re...,67,32,35,0.478,2.12,0.251,2.057,1.938
